In [51]:
from nanotag.data import ImageFileCollection
from nanotag.image import ImageSeries, GaussianFilterSlider
from nanotag.canvas import Canvas, ToolBox
from nanotag.tags import PointTags, PointTagSeries
from nanotag.timeline import Timeline, TimelineTags
from traitlets import directional_link
from nanotag.utils import link, redirected_path
import ipywidgets as widgets
from nanotag.data import NanotagData
from nanotag.data import NanotagData, ImageFileCollection, Summary
import os
from pathlib import Path
from bqplot import ColorScale
from nanotag.histogram import Histogram
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import numpy as np
from collections import defaultdict
import os
import json
from nanotag.tools import EditPointTags, ResetView, BoxZoomTool, PanZoomTool
import glob

def parse_metadata(metadata):
    metadata = metadata['ImageDescription']
    metadata = metadata[metadata.find('{'):]
    metadata = json.loads(metadata)
    return metadata

def parse_voa_current(metadata):
    try:
        timeseries = parse_metadata(metadata)['properties']['timeseries'] 
        return [t['VOA_current'] for t in timeseries]
    except KeyError:
        return 'Not found'


def parse_timestamp(metadata):
    try:
        timeseries = parse_metadata(metadata)['properties']['timeseries'] 
        return [t['timestamp'] for t in timeseries]
    except KeyError:
        return 'Not found'

In [55]:
#root_directory = 'D:/data/julia'
root_directory = '/data'
#root_directory = '/Users/jacobmadsen/nanotag/examples/mos2'

filt = '**/*.tif'

paths = glob.glob(os.path.join(root_directory, os.path.join('analysis', os.path.splitext(filt)[0] + '.json')), recursive=True)

hashes = {}
for path in paths:
    with open(path, 'r') as f:
        data = json.load(f)
        hashes[data['hash']] = os.path.join(os.path.split(os.path.split(path)[0])[-1], os.path.split(path)[-1])

In [56]:
atom_tags = PointTagSeries(data_fields=('col_id','sublattice','intensities','mask'), enable_move=False, visible=False)
is_valid_tags = PointTagSeries(data_fields=('intensities','sublattice'), enable_move=False)
defect_tags = PointTagSeries(data_fields=('type',))
include_tags = TimelineTags(row='include', data_fields=('include',), enable_move=False, color_scale=ColorScale(min=0, max=1))
area_tags = TimelineTags(row='area', data_fields=('area',), enable_move=False, color_scale=ColorScale(min=0, max=1))
edited_tags = TimelineTags(row='edited', data_fields=('edited',), enable_move=False, color_scale=ColorScale(min=0, max=1))

image_series = ImageSeries()
image_file_collection = ImageFileCollection(filter=filt, image_series=image_series)
image_file_collection.root_directory = root_directory

canvas = Canvas()
gaussian_filter = GaussianFilterSlider(max=10)
image_series.filters = [gaussian_filter]
canvas.images = [image_series]

canvas.tags = [atom_tags, is_valid_tags, defect_tags]


toolbox = ToolBox(canvas)
toolbox.tools = {'Reset' : ResetView(),
                 'BoxZoom' : BoxZoomTool(),
                 'Pan' : PanZoomTool(),
                 'S defect' : EditPointTags(defect_tags, data_fields={'type':0}),
                 'S2 defect' : EditPointTags(defect_tags, data_fields={'type':1}),
                 'Mo defect' : EditPointTags(defect_tags, data_fields={'type':2}),
                }

nanotag_data = NanotagData(root_directory = root_directory,
                           read_file = 'no_file.json',
                           tags = {'points' : atom_tags, 'area':area_tags},
                           data_fields = ['shifts', 'included']
                           )

histogram1 = Histogram(markers=('S','S2'), height=220, min=0., max=1.5, bins=40, label='S sublattice', adjust_x_scale=False)
histogram2 = Histogram(markers=('Mo',), height=220, min=.0, max=1.5, bins=40, label='Mo sublattice', adjust_x_scale=False)

def transform_analysis_path(x):
    #x = redirected_path(nanotag_data.root_directory, x, 'analysis')
    x = os.path.join(nanotag_data.root_directory, hashes[image_file_collection.hash])
    #x = os.path.splitext(x)[0] + '.json'
    #x = str(Path(x))
    return x


directional_link((image_file_collection, 'path'), (nanotag_data, 'read_file'), transform_analysis_path)

timeline = Timeline()
timeline.tags = [include_tags, area_tags, edited_tags]

directional_link((timeline, 'frame_index'), (image_series, 'frame_index'))
directional_link((timeline, 'frame_index'), (atom_tags, 'frame_index'))
directional_link((timeline, 'frame_index'), (is_valid_tags, 'frame_index'))
directional_link((timeline, 'frame_index'), (defect_tags, 'frame_index'))

directional_link((nanotag_data, 'shifts'), (image_series, 'shifts'))
link((image_series, 'num_frames'), (timeline, 'num_frames'))

canvas.x_scale.min=-100
canvas.x_scale.max=600

canvas.y_scale.min=-100
canvas.y_scale.max=600

def update_histograms(*args):
    intensities1 = is_valid_tags.point_tags.intensities[is_valid_tags.point_tags.sublattice == 0]
    intensities2 = is_valid_tags.point_tags.intensities[is_valid_tags.point_tags.sublattice == 1]
    histogram1.sample = intensities1
    histogram2.sample = intensities2


def create_frame_summary(i):
    summary = {}
    summary['frame_index'] = int(i)
    summary['lattice_area_fraction'] = 0.
    summary['defect_type'] = []
    summary['defect_x'] = []
    summary['defect_y'] = []
    
    #if len(area_tags.area) == 0:
        #return summary
        
    #print(image_file_collection.metadata)
    #print(parse_metadata(image_file_collection.metadata))
    
    
    #summary['lattice_area_fraction'] = area_tags.area[i]
    
    try:
        summary['sampling'] = nanotag_data.data['sampling']
    except KeyError:
        summary['sampling'] = None
    
    if not i in defect_tags.series.keys():
        return summary
    
    types = defect_tags.series[i]['type']
    mapping = {0:'S',1:'S2',2:'Mo'}
    
    summary['defect_type'] = [mapping[t] for t in types]
    summary['defect_x'] = defect_tags.series[i]['x']
    summary['defect_y'] = defect_tags.series[i]['y']
    summary['voa'] = parse_voa_current(image_file_collection.metadata)[i]
    summary['timestamp'] = parse_timestamp(image_file_collection.metadata)[i]
    return summary

def create_series_summary():
    try:
        summary = {}
        #summary['md5'] = image_file_collection.hash
        summary['num_frames'] = image_series.num_frames
        summary['shape_x'] = image_series.images.shape[1]
        summary['shape_y'] = image_series.images.shape[2]
        summary['energy'] = parse_metadata(image_file_collection.metadata)['properties']['autostem']['high_tension_v']

        #summary['first_frame'] = image_series.images.shape[2]

        if len(include_tags.t) > 0:
            summary['first_frame'] = int(include_tags.t[0])
            summary['last_frame'] = int(include_tags.t[-1])

            first_valid = is_valid_tags.series[include_tags.t[0]]
            summary['included_Mo_columns'] = int(np.sum(np.array(first_valid['sublattice']) == 0))
            summary['included_S_columns'] = int(np.sum(np.array(first_valid['sublattice']) == 1))

            bond_length = 3.15 / np.sqrt(3)

            area_per_columns = 3 * np.sqrt(3) / 4 * bond_length ** 2
            area = (summary['included_Mo_columns'] + summary['included_S_columns']) * area_per_columns
            summary['area'] = area

            #summary['voa'] = parse_voa_current(image_file_collection.metadata)[summary['first_frame']:summary['last_frame']]

        summary['normalized_S2_threshold'] = histogram1.S2
        summary['normalized_S_threshold'] = histogram1.S
        summary['normalized_Mo_threshold'] = histogram2.Mo
        summary['path'] = image_file_collection.relative_path
    except:
        summary = {}
    
    #if i is None:
    
    #else:
    #    summary_data['voa'] = parse_voa_current(image_file_collection.metadata)[:i + 1]
    
    return summary

def update_summary(*args):
    current_data = create_series_summary()
    current_data.update(create_frame_summary(timeline.frame_index))
    
    data = create_series_summary()
    for i in include_tags.t:
        data[int(i)] = create_frame_summary(i)
    
    summary.current_data = current_data
    summary.data = data
    
defect_tags.point_tags.observe(update_summary, 'x')
    
def new_valid(*args):
    data = defaultdict(list)
    for frame_id in include_tags.t:
        frame_id = str(frame_id)
        
        if not str(frame_id) in nanotag_data.data['points'].keys():
            continue
        
        values = nanotag_data.data['points'][frame_id]
    
        for i, l in enumerate(values['col_id']):
            data[l].append(values['mask'][i])
    
    included = []
    for col_id, values in data.items():
        if np.all(np.array(values) == 0.):
            included.append(col_id)
    
    included_series = {}
    for frame_id in include_tags.t:
        frame_id = str(frame_id)
        
        if not str(frame_id) in nanotag_data.data['points'].keys():
            continue
        
        values = nanotag_data.data['points'][frame_id]
        
        a = np.array(values['col_id'])[:, None] == np.array(included)[None]
        i = np.where(np.any(a, axis=1))[0]
        
        included_series[int(frame_id)] = {}
        included_series[int(frame_id)]['x'] = list(np.array(values['x'])[i])
        included_series[int(frame_id)]['y'] = list(np.array(values['y'])[i])
        included_series[int(frame_id)]['sublattice'] = list(np.array(values['sublattice'])[i])
        included_series[int(frame_id)]['intensities'] = list(np.array(values['intensities'])[i])
        
    is_valid_tags.series = included_series
    
    
def new_series_callback(*args):
    include_tags.t = np.where(area_tags.area > .5)[0]
    if len(include_tags.t) > 0:
        timeline.frame_index = int(include_tags.t[0])
    else:
        timeline.frame_index = 0
        
    new_valid()

    
def defect_threshold_changed(*args):
    series = {}
    for key, value in is_valid_tags.series.items():
        
        if np.any(key == np.array(edited_tags.t)):
            series[key] = defect_tags.series[key]
            continue
        
        series[key] = {'x':[], 'y':[], 'type':[]}
        
        if len(value['x']) == 0:
            continue
        
        x = np.array(value['x'])
        y = np.array(value['y'])
        sublattice = np.array(value['sublattice'])
        intensities = np.array(value['intensities'])
        
        intensities1 = intensities[sublattice == 0]
        intensities2 = intensities[sublattice == 1]
        
        S2_thres = min(histogram1.S2, histogram1.S)
        S_thres = max(histogram1.S2, histogram1.S)
        
        S2 = (intensities1 < S2_thres)
        S = (intensities1 < S_thres) * (intensities1 > S2_thres)
        Mo = (intensities2 < histogram2.Mo)
        
        x = np.concatenate((x[sublattice == 0][S], x[sublattice == 0][S2], x[sublattice == 1][Mo]))
        y = np.concatenate((y[sublattice == 0][S], y[sublattice == 0][S2], y[sublattice == 1][Mo]))
        
        if len(x) == 0:
            continue
        
        types = [0] * sum(S) + [1] * sum(S2) + [2] * sum(Mo)
        series[key]['x'] = x.tolist()
        series[key]['y'] = y.tolist()
        series[key]['type'] = types
    
    defect_tags.series = series
    update_summary()
    
def new_frame_callback(*args):
    edited_tags.t = []
    update_histograms()
    defect_threshold_changed()
    create_series_summary()

nanotag_data.observe(new_series_callback, 'data')
area_tags.observe(new_series_callback, 'area')

atom_tags.point_tags.observe(new_frame_callback, 'y')
is_valid_tags.point_tags.observe(new_frame_callback, 'y')

histogram1.observe(defect_threshold_changed, 'S')
histogram1.observe(defect_threshold_changed, 'S2')
histogram2.observe(defect_threshold_changed, 'Mo')

atom_tags.point_tags.visible = False
defect_tags.point_tags.color_scheme = 'category'
defect_tags.point_tags.color_scale.min=0
defect_tags.point_tags.color_scale.max=10

def set_first_frame(*args):
    include_tags.t = list(range(timeline.frame_index, include_tags.t[-1] + 1))
    new_valid()

def set_last_frame(*args):
    include_tags.t = list(range(include_tags.t[0], timeline.frame_index + 1))
    new_valid()

def transform_summary_path(x):
    if x is None:
        return ''
    
    return os.path.splitext(x)[0] + '.json'

def edit(*args):
    edited_tags.t = np.sort(np.concatenate((edited_tags.t, [timeline.frame_index])))
    update_summary()
    defect_tags.serialize()
    
def remove_edit(*args):
    edited_tags.t = edited_tags.t[np.array(edited_tags.t) != timeline.frame_index]
    update_summary()
    defect_threshold_changed()
    
    
edit_button = widgets.Button(description='Edit')

remove_edit_button = widgets.Button(description='Remove edits')

edit_button.on_click(edit)
remove_edit_button.on_click(remove_edit)

edits_box = widgets.HBox([edit_button, remove_edit_button])

first_frame = widgets.Button(description='First frame')
last_frame = widgets.Button(description='Last frame')

first_frame.on_click(set_first_frame)
last_frame.on_click(set_last_frame)

summary = Summary(write_file='summary.json')

directional_link((image_file_collection, 'path'), (summary, 'write_file'), transform_summary_path);

TraitError: The 'path' trait of an ImageFileCollection instance expected a unicode string, not the NoneType None.

In [57]:
toolbox.button_width = '78px'

tools_box = widgets.VBox([nanotag_data, image_file_collection, gaussian_filter, image_series, is_valid_tags, defect_tags, toolbox, edits_box])

hisogram_box = widgets.VBox([histogram1, histogram2])

tabs = widgets.Tab([tools_box, hisogram_box])
tabs.set_title(0, 'Tools')
tabs.set_title(1, 'Histograms')

from nanotag.events import KeyEvents


def toggle_point_visible(*args):
    is_valid_tags.point_tags.visible = not is_valid_tags.point_tags.visible

def toggle_defects_visible(*args):
    defect_tags.point_tags.visible = not defect_tags.point_tags.visible

    

canvas_box = widgets.VBox([canvas, timeline, widgets.HBox([first_frame, last_frame])])

events = KeyEvents(canvas_box)

events.callbacks = {'ArrowLeft' : timeline.previous_frame,
                    'ArrowRight' : timeline.next_frame,
                    'ArrowUp' : image_file_collection.next_file,
                    'ArrowDown' : image_file_collection.previous_file,
                    'r' : canvas.reset,
                    'z' : lambda : toolbox.toggle_tool('BoxZoom'),
                    'p' : lambda : toolbox.toggle_tool('Pan'),
                    'l' : toggle_point_visible,
                    'd' : toggle_defects_visible,
                   }


new_series_callback()
new_frame_callback()
defect_threshold_changed()

app = widgets.HBox([canvas_box, tabs, summary])

app